# Day 13: Vertical Federated Learning

**Feature-Partitioned Federated Learning**

## Overview
- **Scenario**: Different parties have different FEATURES for the same samples
- **Example**: Bank + Credit Bureau collaborating on loan approval
- **Challenge**: How to train without sharing raw features?

## What You'll Learn
1. **Vertical FL vs Horizontal FL**: Feature partitioning vs sample partitioning
2. **Entity Alignment**: Matching samples across parties
3. **Split Learning**: Model partitioning across parties
4. **Use Cases**: Real-world vertical FL scenarios

---

## 1. Horizontal vs Vertical FL

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("""
╔══════════════════════════════════════════════════════════════════════════╗
║                        FEDERATED LEARNING TYPES                          ║
╚══════════════════════════════════════════════════════════════════════════╝

HORIZONTAL FL (Days 8-12):
  • Same features, different samples
  • Example: Regional banks with different customers
  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐
  │  Bank A      │  │  Bank B      │  │  Bank C      │
  │  ┌────────┐  │  │  ┌────────┐  │  │  ┌────────┐  │
  │  │User 1  │  │  │  │User 4  │  │  │  │User 7  │  │
  │  │User 2  │  │  │  │User 5  │  │  │  │User 8  │  │
  │  │User 3  │  │  │  │User 6  │  │  │  │User 9  │  │
  │  └────────┘  │  │  └────────┘  │  │  └────────┘  │
  │  Features:   │  │  Features:   │  │  Features:   │
  │  [amount,    │  │  [amount,    │  │  [amount,    │
  │   time,      │  │   time,      │  │   time,      │
  │   merchant]  │  │   merchant]  │  │   merchant]  │
  └──────────────┘  └──────────────┘  └──────────────┘

VERTICAL FL (Today):
  • Same samples, different features
  • Example: Bank + Credit Bureau
  ┌──────────────┐              ┌──────────────┐
  │    Bank      │              │ Credit Bureau│
  │  ┌────────┐  │              │  ┌────────┐  │
  │  │User 1  │  │◄────────────►│  │User 1  │  │
  │  │User 2  │  │              │  │User 2  │  │
  │  │User 3  │  │◄────────────►│  │User 3  │  │
  │  └────────┘  │              │  └────────┘  │
  │  Features:   │              │  Features:   │
  │  [amount,    │              │  [credit     │
  │   balance,   │              │   score,     │
  │   income]    │              │   debt,      │
  │              │              │   history]   │
  └──────────────┘              └──────────────┘

""")

print("Key Difference:")
print("  Horizontal: Different ROWS (samples), same COLUMNS (features)")
print("  Vertical:    Same ROWS (samples), different COLUMNS (features)")

## 2. Vertical FL Scenario: Fraud Detection

In [ ]:
# Scenario: Bank + Payment Processor + Credit Bureau
print("""

VERTICAL FRAUD DETECTION SCENARIO:

┌─────────────────────────────────────────────────────────────────────┐
│                     SAME TRANSACTION                                │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  ┌───────────────┐    ┌──────────────────┐    ┌─────────────────┐   │
│  │     BANK      │    │ PAYMENT PROCESSOR│    │ CREDIT BUREAU   │   │
│  │               │    │                  │    │                 │   │
│  │ Features:     │    │ Features:        │    │ Features:       │   │
│  │ • Amount      │    │ • Device ID      │    │ • Credit Score  │   │
│  │ • Account     │    │ • IP Address     │    │ • Debt Ratio    │   │
│  │   Balance     │    │ • Location       │    │ • Payment Hist  │   │
│  │ • Customer    │    │ • Browser        │    │ • Account Age   │   │
│  │   Segment     │    │ • OS             │    │ • Inquiries     │   │
│  │ • Income      │    │ • Timestamp      │    │                 │   │
│  │               │    │                  │    │                 │   │
│  └───────────────┘    └──────────────────┘    └─────────────────┘   │
│           │                     │                      │            │
│           └─────────────────────┼──────────────────────┘            │
│                                 ▼                                   │
│                    ┌─────────────────────────┐                      │
│                    │  VERTICAL FL MODEL      │                      │
│                    │                         │                      │
│                    │  [Combine features]    │                      │
│                    │  [Predict fraud]       │                      │
│                    └─────────────────────────┘                      │
└─────────────────────────────────────────────────────────────────────┘

Challenge: No party sees the others' raw features!

""")

## 3. Entity Alignment

In [ ]:
import pandas as pd
import numpy as np

# Sample data from two parties
bank_data = pd.DataFrame({
    'customer_id': ['A101', 'B205', 'C307', 'D412', 'E519'],
    'transaction_amount': [1500, 300, 5000, 200, 8000],
    'account_balance': [5000, 12000, 800, 3000, 1500]
})

bureau_data = pd.DataFrame({
    'customer_id': ['A101', 'C307', 'F623', 'B205', 'E519'],
    'credit_score': [720, 680, 750, 710, 690],
    'debt_ratio': [0.3, 0.45, 0.2, 0.35, 0.5]
})

print("Party A (Bank) Data:")
print(bank_data)
print("\nParty B (Credit Bureau) Data:")
print(bureau_data)

# Entity alignment
aligned_data = pd.merge(
    bank_data,
    bureau_data,
    on='customer_id',
    how='inner'  # Only keep customers in BOTH datasets
)

print("\n" + "="*60)
print("ENTITY ALIGNMENT RESULT:")
print("="*60)
print(aligned_data)
print(f"\nOriginal: Bank={len(bank_data)}, Bureau={len(bureau_data)}")
print(f"Aligned (intersection): {len(aligned_data)} customers")
print(f"Lost: {len(bank_data) + len(bureau_data) - 2*len(aligned_data)} customers")

## 4. Split Learning Approach

In [ ]:
print("""

SPLIT LEARNING FOR VERTICAL FL:

Idea: Split the neural network across parties

┌─────────────────────────────────────────────────────────────────────┐
│                         SPLIT NEURAL NETWORK                         │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  ┌───────────────────┐              ┌───────────────────┐           │
│  │     BANK SIDE     │              │  BUREAU SIDE      │           │
│  ├───────────────────┤              ├───────────────────┤           │
│  │ Input:            │              │ Input:            │           │
│  │  [amount, bal]    │              │  [credit, debt]   │           │
│  │                   │              │                   │           │
│  │ ┌───────────────┐ │              │ ┌───────────────┐ │           │
│  │ │ Dense Layer 1│ │              │ │ Dense Layer 1 │ │           │
│  │ │ (hidden=32)   │ │              │ │ (hidden=32)   │ │           │
│  │ └───────────────┘ │              │ └───────────────┘ │           │
│  │                   │              │                   │           │
│  │ ┌───────────────┐ │              │ ┌───────────────┐ │           │
│  │ │ Dense Layer 2│ │              │ │ Dense Layer 2 │ │           │
│  │ │ (hidden=16)   │ │              │ │ (hidden=16)   │ │           │
│  │ └───────────────┘ │              │ └───────────────┘ │           │
│  │                   │              │                   │           │
│  └─────────┬─────────┘              └─────────┬─────────┘           │
│            │                                  │                      │
│            │  Embeddings (no raw features!)   │                      │
│            └──────────────┬───────────────────┘                      │
│                           ▼                                         │
│                ┌─────────────────────────┐                           │
│                │   TRUSTED SERVER /       │                           │
│                │   SECURE AGGREGATION     │                           │
│                ├─────────────────────────┤                           │
│                │                         │                           │
│                │  Concatenate embeddings │                           │
│                │  ┌───────────────────┐  │                           │
│                │  │ Dense Layer 3    │  │                           │
│                │  │ (hidden=32)       │  │                           │
│                │  └───────────────────┘  │                           │
│                │                         │                           │
│                │  ┌───────────────────┐  │                           │
│                │  │ Output Layer      │  │                           │
│                │  │ (sigmoid)         │  │                           │
│                │  └───────────────────┘  │                           │
│                │                         │                           │
│                │  Prediction: Fraud?    │                           │
│                └─────────────────────────┘                           │
└─────────────────────────────────────────────────────────────────────┘

Key Points:
  1. Each party trains on their own features
  2. Only EMBEDDINGS are shared (not raw data)
  3. Trusted server combines embeddings for final prediction
  4. Gradient backpropagation works across the split

""")

## 5. Privacy-Preserving Techniques

In [ ]:
techniques_df = pd.DataFrame({
    'Technique': [
        'Split Learning',
        'Homomorphic Encryption',
        'Secure Multi-Party Computation',
        'Feature Exchange with Noise'
    ],
    'Description': [
        'Split model, share intermediate embeddings',
        'Compute on encrypted data (very slow)',
        'Distributed computation with crypto',
        'Add differential privacy noise to features'
    ],
    'Pros': [
        '✅ Conceptually simple, works today',
        '✅ Strong privacy guarantee',
        '✅ No raw data shared',
        '✅ Easy to implement'
    ],
    'Cons': [
        '❌ Requires trusted server, slower',
        '❌ Extremely slow (1000x slowdown)',
        '❌ High communication cost',
        '❌ Reduces model accuracy'
    ]
})

print("\n" + "="*80)
print("VERTICAL FL PRIVACY TECHNIQUES")
print("="*80)
print(techniques_df.to_string(index=False))

## 6. Summary

### Vertical FL Key Points:

**When to Use Vertical FL:**
- Multiple parties have DIFFERENT FEATURES for same entities
- Example: Bank + Credit Bureau + Payment Processor
- Feature complementarity improves model performance

**Key Challenges:**
- Entity alignment (matching samples across parties)
- Secure aggregation (how to combine features without sharing)
- Communication overhead (more expensive than horizontal FL)

**Approaches:**
1. **Split Learning**: Most practical, used in production
2. **Homomorphic Encryption**: Strong privacy, very slow
3. **SMPC**: Theoretical interest, high overhead

**Use Cases:**
- Fraud detection (bank + payment processor)
- Credit scoring (bank + credit bureau)
- Healthcare (hospital + insurance company)
- Recommendation (content provider + social network)

### Next Steps:
→ **Day 14**: Label Flipping Attack (adversarial threats)

---

**📁 Project Location**: `02_federated_learning_foundations/vertical_fraud_detection/`